# Структуры

## Зачем группировать данные?

Какая должна быть сигнатура у функции, которая вычисляет длину отрезка на плоскости?

    double  length(double x1, double y1 ,
                   double x2, double  y2);

А сигнатура функции, проверяющей пересечение отрезков? 

    bool  intersects(double x11 , double y11 ,
                     double x12 , double y12 ,
                     double x21 , double y21 ,
                     double x22 , double y22 ,
                     double * xi , double * yi);

Координаты точек являются логически связанными данными,которые всегда передаются вместе.  
Аналогично связанны координаты точек отрезка.

## Структуры

Структуры — это способ синтаксически (и физически) сгруппировать логически связанные данные.  

    struct  Point {         // в памяти - два подряд значения типа double
        double x;
        double y;
    };

    struct  Segment {       // в памяти - два подряд значения типа Point
        Point p1;
        Point p2;
    };                      // ; обязательно для структур
    
    double  length(Segment s);

    bool  intersects(Segment s1,        // теперь сигнатура вполне читабельная
                     Segment s2, 
                     Point * p);        // указатель на Point

\*) в памяти поля структур в общем случае (без специальных директив) выравниваются кратно своему размеру, но не менее 4 байт (?). Например, поля char + int в памяти будут занимать 1 байт (чар), 3 пустых байта, 4 байта (инт) = 8 байт. Но более длинные последовательности зачастую располагаются компилятором всякими причудливыми образами, выравнивания могут идти неподряд, типа 2 шорта + 2 байта пустых и т.п.  
Есть директива, принудительно устанавливающая размер выравнивания (например, 1 байт, т.е. вплотную):

    #pragma pack(push, 1)
    struct Foo
    {
        // ...
    };
    #pragma pack(pop)   // обязательно отключать сразу !

Полезно для упаковки структур с битовыми полями (NB: зависят от порядка байтов, при порядке LITTLE_ENDIAN битовые поля раздаются начиная со первых байтов, при BIG_ENDIAN — наоборот).

NB: всегда тестировать размер структуры sizeof(), чтобы понять, как она по факту упаковалась компилятором:
    
    #pragma pack(push,1)
    struct MyBitStruct
    {
        uint16_t a:4;
        uint16_t b:4;
        uint16_t c;
    };
    #pragma pack(pop)

Получилась структура на 4 байта! Две половины первого байта — это поля a и b. Второй байт не доступен по имени и последние 2 байта доступны по имени c

+ методы в структуре не влияют на sizeof()



## Работа со структурами

Доступ к полям структуры осуществляется через оператор ’.’:

    #include  <cmath >                      // sqrt

    double  length(Segment s) {             // недостаток в том, что значения структуры Segment копируются для передачи в функцию
        double  dx = s.p1.x - s.p2.x;
        double  dy = s.p1.y - s.p2.y;
        return  sqrt(dx * dx + dy * dy);
    }

Для указателей на структуры используется оператор ’->’.

    double  length(Segment * s) {           // если передаем указатель на структуру 
        double  dx = s->p1.x - s->p2.x;     // то обращаться к полям структуры через ->
        double  dy = s->p1.y - s->p2.y;     // другой подход (менее красивый) - разыменовать указатель: double  dy = (*s).p1.y - (*s).p2.y
        return  sqrt(dx * dx + dy * dy);
    }

## Инициализация структур

Поля структур можно инициализировать подобно массивам:

    Point p1   = { 0.4, 1.4 };          // синтаксис похож на массивы, порядок полей - как в объявлении структуры
    Point p2   = { 1.2, 6.3 };
    Point p3     { 9.1, 1.3 };          // uniform-инициализация (C++11)

    Segment s  = { p1, p2   };

Структуры могут хранить переменные разных типов.

    struct  IntArray2D {
        size_t a;
        size_t b;
        int ** data;
    };
    
    // экземпляр структуры
    IntArray2D a = {n, m, create_array2d(n, m)};    // при инициализации таких структур можно использовать внешний "конструктор"

# Методы

## Методы

Метод — это функция, определённая внутри структуры.
    
    struct  Segment {
        Point p1;
        Point p2;
        double  length () {
            double  dx = p1.x - p2.x;
            double  dy = p1.y - p2.y;
            return  sqrt(dx * dx + dy * dy);
        }
    };
    
    int  main() {
        Segment s = { { 0.4, 1.4 }, { 1.2, 6.3 } };
        cout  << s.length () << endl;
        return  0;
    }

Методы реализованы как функции с неявным параметром this, который указывает на текущий экземпляр структуры.
    
    struct  Point{
        double x;
        double y;
        void  shift(/*  Point * this , */
                    double x, 
                    double y) {
                        this ->x += x;
                        this ->y += y;
        }
    };


## Методы: объявление и определение

Методы можно разделять на объявление и определение:

    struct  Point{
        double x;
        double y;
        void  shift(double x, double y);
    };

    void  Point::shift(double x, double y) {
        this ->x += x;
        this ->y += y;
    }

## Абстракция (независимость от конкретных данных) и инкапсуляция (объединение данных и методов)

Использование методов позволяет объединить данные и функции для работы с ними.

    struct  IntArray2D {
        int & get(size_t i, size_t j) {     // метод перевода одномерного массива в 2d
            return  data[i * b + j];        // блоками размера this.b
        }
        size_t a;
        size_t b;
        int * data;                         // одномерный массив
    };
    
    IntArray2D m = foo();
    for (size_t i = 0; i != m.a; ++i )
        for (size_t j = 0; j != m.b; ++j)
            if (m.get(i, j) < 0) 
                m.get(i,j) = 0;

### Структуры существуют только до момента компиляции, непосредственно в ассмеблерном коде они никак отражены не будут (см. /src)
#### - это означает, что дальнейшая работа со структурами идет на уровне на уровне байт, без информации о конкретных типах

# Конструкторы и деструкторы

## Конструкторы

Конструкторы — это методы для инициализации структур.  
Синтаксически - это функция с именем структуры, которая не возвращает ничего (даже void).   
Конструкторов может быть несколько.

    struct  Point {
        Point() {                           // конструктор 1 (без параметров, просто инициализация)
            x = y = 0;                      // синтаксически так можно => x = (y = 0) => x = y (выражение в скобках возвращает lvalue)
        }                
        Point(double x, double y) {         // конструктор 2 (с параметрами)
            this->x = x;
            this->y = y;
        }
        double x;
        double y;
    };
    
    Point p1;                               // {0, 0} => по умолчанию - конструктор без параметров
    Point p2(3 ,7);

Структуры как и классы могут содержать конструкторы. Для структур установлены следующие правила применения конструкторов:

    структура может иметь любое количество конструкторов, которые отличаются типами параметров и/или количеством параметров;
    структура может содержать конструкторы, которые объявлены в разделах доступа: private, public, protected;
    если в структуре нет ни одного конструктора, то переменная (экземпляр) типа структуры создается обычным способом без указания параметров;
    если в структуре объявлен общедоступный конструктор, то экземпляр структуры должен обязательно создаваться с параметрами, которые соответствуют этому конструктору.


## Список инициализации

Список инициализации позволяет проинициализировать поля до входа в конструктор.  
Синтаксис такого списка:  

Имя конструктора : поле1(начальный параметр), поле2(начальный параметр), ...   
// формально, писать поля в списке можно в произвольном порядке, но инициализироваться они будут все равно в порядке объявления, что чревато ошибками

    struct  Point {
        Point() : x(0), y(0)
        {}
        Point(double x, double y) : x(x), y(y)
        {}
        
        double x;
        double y;
    };

Инициализации полей в списке инициализации происходит впорядке объявления полей в структуре.

## Значения по умолчанию

∙Любые функции в С++ могут иметь значения параметров по умолчанию.  
∙Значения параметров по умолчанию нужно указывать в объявлении функции.

    struct  Point {
        Point(double x = 0, double y = 0)       // можно заменить на 1 конструктор со значениями по умолчанию
            : x(x), y(y)
        {}
        double x;
        double y;
    };
    
    Point p1;
    Point p2(2);                                // порядковый аргумент (х)
    Point p3(3,4);

## Конструкторы от одного параметра

Конструкторы от одного параметра задают неявное пользовательское преобразование:
    
    struct  Segment {
        Segment () {}               // т.к. у полей есть свои конструкторы, то даже без параметров тут, они проинициализируются там
        Segment(double  length)
            : p2(length , 0)        // используется конструктор поля Point p2
        {}
        Point p1;
        Point p2;
    };
    
    Segment  s1;                    // (0,0) (0,0)
    Segment  s2(10);                // (0,0) (10,0)
    Segment  s3 = 20;               // эквив. инициализации числом 20 (s3 = Serment(20)), 
                                    // т.к. такой конструктор от 1 параметра есть, получаем неявно преобразование (0,0) (20,0)
                                    // чревато ошибками...

Для того, чтобы запретить неявное пользовательское преобразование, используется ключевое слово explicit.

    struct Segment {
        Segment () {}
        explicit Segment(double length)     // разрешает компилятору только явный вызов конструктора
            : p2(length , 0)
        {}
        Point p1;
        Point p2;
    };
    
    Segment s1;
    Segment s2(10);
    Segment s3 = 20; // error

Неявное пользовательское преобразование, задаётся также конструкторами, которые могут принимать один параметр.

    struct  Point {
        explicit  Point(double x = 0, double y = 0)     // тут тоже имеет смысл защититься от неявных вызовов конструктора
            : x(x), y(y)
        {}
        double x;
        double y;
    };
    
    Point p1;
    Point p2(2);
    Point p3(3 ,4);
    Point p4 = 5; // error

## Конструктор по умолчанию

Если у структуры нет конструкторов, то конструктор без параметров, он же конструктор по умолчанию (default constructor), генерируется компилятором.

    struct  Segment {
        Segment(Point p1 , Point  p2)       // только 1 конструктов, кот. по 2 точкам создает отрезок
            : p1(p1), p2(p2)                // а конструктора без параметров не предусмотрено
        {}
        Point p1;
        Point p2;
    };
    
    Segment  s1;                            // error (т.к. нет подходящего конструктора (который без параметров))
    Segment  s2(Point(), Point (2 ,1));     // нужен явный вызов конструкторов полей => (0,0) (2,1)

## Особенности синтаксиса C++

“Если что-то похоже на объявление функции, то это и есть объявление функции.”

    struct  Point {
        explicit  Point(double x = 0, double y = 0)     // один конструктор в роли конструктора по умолчанию (т.е. конструктор без аргументов), с 1 и 2 аргументами
            : x(x), y(y) 
        {}
        double x;
        double y;
    };
    
    Point p1;           // ОК определение переменной типа Point
    Point p2();         // не ОК, попытка явно вызвать конст.по умолчанию оказывается похожа на объявление функции
                        // так она и будет скомпилирована (как объявление функции в возвращаемым значением типа Point)
    
    double k = 5.1;
    Point p3(int(k));   // не ОК, будет расценено как объявление функции, тонкий момент в том, что int(k) хоть и 
                        // это допустимый синтаксис для оператора приведения типа, но тут будет принято за объявление
                        // переменной int k, а вся конструкция - за объявление функции

    Point p4((int)k);   // ОК, определение переменной

## Деструктор

Деструктор — это метод, который вызывается при удалении структуры, по умолчанию они генерируется компилятором.  
Синтаксически это функция, имя которой совпадает с именем структуры, но начинается с ~.  
Может быть только один.

        struct  IntArray {
            explicit  IntArray(size_t  size)
                : size(size)
                , data(new int[size])
            { }
            
            ~IntArray () {
                delete  [] data;
            }
            
            size_t  size;
            int *   data;
        };

## Время жизни

Время жизни — это временной интервал между вызовами конструктора и деструктора.

    void  foo()
    {
        IntArray  a1 (10);      // создание a1
        IntArray  a2 (20);      // создание a2
        for (size_t i = 0; i != a1.size; ++i) {
            IntArray  a3 (30);  // создание a3
            ...
        } // удаление a3
    } // удаление a2, потом a1

Деструкторы переменных на стеке вызываются в обратном порядке (по отношению к порядку вызова конструкторов).

## Идиома программирования RAII

Идиома RAII (англ. «Resource Acquisition Is Initialization» = «Получение ресурсов есть инициализация») — это идиома объектно-ориентированного программирования, при которой использование ресурсов привязывается к времени жизни объектов с автоматической продолжительностью жизни. В языке C++ идиома RAII реализуется через классы с конструкторами и деструкторами. Ресурс (например, память, файл или база данных) обычно приобретается в конструкторе объекта (хотя этот ресурс может быть получен и после создания объекта, если в этом есть смысл). Затем этот ресурс можно использовать, пока объект жив. Ресурс освобождается в деструкторе при уничтожении объекта. Основным преимуществом RAII является то, что это помогает предотвратить утечку ресурсов (например, памяти, которая не была освобождена), так как все объекты, содержащие ресурсы, автоматически очищаются.

# Объекты и классы

### Структуру с методами, конструкторами и деструктором называют классом.  

∙Экземпляр (значение) класса называется объектом.

    struct  IntArray {
        explicit  IntArray(size_t  size);
        ~IntArray ();
        int & get(size_t i);

        size_t  size;
        int *   data;
    };
    
    IntArray a(10);
    IntArray b = {20, new int [20]};  // ошибка (если есть констукторы/деструктор, то объект уже не ведет себя как структура)

## Объекты в динамической памяти. Создание

Для создания объекта в динамической памяти используется оператор new, он отвечает за вызов конструктора.

    struct  IntArray {
        explicit  IntArray(size_t  size);
        ~IntArray ();
        
        size_t  size;
        int *   data;
    };
    
    // выделение памяти и создание объекта
    IntArray * pa = new  IntArray (10);                 
    // только выделение памяти
    IntArray * pb = (IntArray  *) malloc(sizeof(IntArray ));        // malloc конструктор не вызывает

## Объекты в динамической памяти. Удаление

При вызове оператора delete вызывается деструктор объекта.

    // выделение памяти и создание объекта
    IntArray * pa = new  IntArray (10);
    
    // вызов деструктора и освобождение памяти
    delete  pa;

Операторы new [] и delete [] работают аналогично

    // выделение памяти и создание массива из 10 объектов
    // (вызывается конструктор по умолчанию)
    IntArray * pa = new  IntArray [10];
    
    // вызов деструкторов и освобождение памяти
    delete  [] pa;

Пример:
        
        const size_t count = 10;

        char *buffer = new char[count * sizeof(int)];
        int *ptr = (int *)buffer;

        for (size_t i = 0; i != count; ++i)
                new (ptr + i) int(i);
        delete[] buffer;

\*) тут используется тип int, поэтому не нужно явно вызывать деструкторы, для более сложных типов это придется сделать.  
Кроме того, можно напрямую вызвать оператор new (я так и предпочитаю делать), а не алоцировать массив char-ов (хотя это вполне легальная конструкция):
        
        int *ptr = (int *) ::operator new[](count * sizeof(int));


## Placement new

Если в программе используется специальная аллокация памяти:

    // выделение памяти
    void * p = myalloc(sizeof(IntArray ));
    
    // создание объекта по адресу p
    IntArray * a = new (p) IntArray (10);       // спец.форма оператора new (new с размещением по указателю p)
    
    // явный вызов деструктора
    a->~IntArray ();                            // вызов как обычный метод
    
    // освобождение памяти
    myfree(p);

Проблемы с выравниванием:  
Компилятор может накладывать органичения на выравнивание адресов. При специальных схемах аллокации за этим надо следить.
    
    char b[sizeof(IntArray )];  // массив на стеке
    new (b) IntArray (20);      // потенциальная проблема

# Модификаторы доступа

Модификаторы доступа позволяют ограничивать доступ к методам и полям класса.

    struct  IntArray {
        explicit  IntArray(size_t  size)
            : size_(size), data_(new int[size])
        {}
        ~IntArray () { delete  []  data_; }
        
        int & get(size_t i) { return  data_[i]; }
        size_t  size()      { return  size_; }
        
    private:
        size_t  size_;
        int *   data_;
    };

## Ключевое слово class

Ключевое слово struct можно заменить на class, тогда поля и методы по умолчанию будут private.  
Т.е. нужно явно указывать, что будет публичным.

    class  IntArray {
    public:
        explicit  IntArray(size_t  size)
            : size_(size), data_(new int[size])
        {}
        ~IntArray () { delete  []  data_;   }
        
        int &   get(size_t i) { return  data_[i]; }
        size_t  size()        { return  size_; }
    
    private:                // никакая внешняя функция не сможет к ним обратиться
        size_t  size_;
        int *   data_;
    };

## Инварианты класса

∙Выделение публичного интерфейса позволяет поддерживать инварианты класса (сохранять данные объекта в согласованном состоянии).
    
    // инвариант класса - тут это значит поле size_ отражает фактический размер массива data_
    struct  IntArray {
        ...
        size_t  size_;
        int *   data_; // массив размера size_
    };

∙Для сохранения инвариантов класса:

    1. все поля должны быть закрытыми,
    2. публичные методы должны сохранять инварианты класса.

∙Закрытие полей класса позволяет абстрагироваться от способа хранения данных объекта.

## Публичный интерфейс 

Корректный интерфейс, который сохраняет инварианты класса при любой работе любоего публичного метода класса.  
Например, не должен один публичный метод менять size, а другой размер data.

    struct  IntArray {
        ...
        void  resize(size_t  nsize) {
            int * ndata = new  int[nsize ];
            size_t n = nsize > size_ ? size_ : nsize;
            for (size_t i = 0; i != n; ++i)
                ndata[i] = data_[i];
            delete  [] data_;
            data_ = ndata;
            size_ = nsize;
        }
    private:
        size_t  size_;
        int *   data_;
    };

## Абстрагирование от деталей реализации

Пример изменения деталей реализации класса без изменения публичного интерфейса.  
Исходный класс:

    struct  IntArray {
    public:
        explicit  IntArray(size_t  size)
            : size_(size), data_(new int[size])
        {}
        ~IntArray () { delete  []  data_;   }
        
        int &   get(size_t i) { return  data_[i]; }
        size_t  size()        { return  size_; }
    
    private:
        size_t  size_;
        int *   data_;
    };

Решили заняться улучшениями. Почему бы не хранить поле size как первый элемент поля data.

    struct  IntArray {
    public:
        explicit  IntArray(size_t  size)
            : data_(new  int[size + 1])                     // выделяем в кунструкторе на 1 эл-т больше
        {
            data_ [0] = size;                               // в 0-й элемент пишем size
        }
        ~IntArray () { delete  []  data_;   }

        int &   get(size_t i) { return  data_[i + 1]; }     // возвращаему ссылку +1
        size_t  size()        { return  data_ [0]; }        // возвращаем size
    private:
        int *   data_;
    };

Сигнатуры интерфейса не изменились, логика работы класса не изменилась, внешние пользователи работают с классом без изменений.  
Оптимизация удалась.